In [13]:
import pandas as pd 
import IPython.display as ipd
import librosa

In [3]:
data = pd.read_csv("ESC-50-master/meta/esc50.csv") 

In [4]:
data.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [5]:
data["category"]

0                    dog
1         chirping_birds
2         vacuum_cleaner
3         vacuum_cleaner
4           thunderstorm
5           thunderstorm
6        door_wood_knock
7            can_opening
8                   crow
9        door_wood_knock
10       door_wood_knock
11              clapping
12              clapping
13              clapping
14                   dog
15              clapping
16          thunderstorm
17             fireworks
18             fireworks
19             fireworks
20             fireworks
21              clapping
22              clapping
23              clapping
24              chainsaw
25              airplane
26           mouse_click
27         pouring_water
28                 train
29                 sheep
              ...       
1970         water_drops
1971      glass_breaking
1972      chirping_birds
1973                 cat
1974         water_drops
1975            laughing
1976    drinking_sipping
1977         can_opening
1978           breathing


In [6]:
dogs = data[data['category']=="dog"]
print(dogs)
len(dogs)

              filename  fold  target category  esc10  src_file take
0     1-100032-A-0.wav     1       0      dog   True    100032    A
14    1-110389-A-0.wav     1       0      dog   True    110389    A
157    1-30226-A-0.wav     1       0      dog   True     30226    A
158    1-30344-A-0.wav     1       0      dog   True     30344    A
170    1-32318-A-0.wav     1       0      dog   True     32318    A
296    1-59513-A-0.wav     1       0      dog   True     59513    A
375    1-85362-A-0.wav     1       0      dog   True     85362    A
392    1-97392-A-0.wav     1       0      dog   True     97392    A
478   2-114280-A-0.wav     2       0      dog   True    114280    A
479   2-114587-A-0.wav     2       0      dog   True    114587    A
482   2-116400-A-0.wav     2       0      dog   True    116400    A
484   2-117271-A-0.wav     2       0      dog   True    117271    A
496   2-118072-A-0.wav     2       0      dog   True    118072    A
503   2-118964-A-0.wav     2       0      dog   

40

In [7]:
data["category"].values

array(['dog', 'chirping_birds', 'vacuum_cleaner', ..., 'footsteps',
       'sheep', 'dog'], dtype=object)

In [8]:
categories = set()
for category in data["category"].values:
    categories.add(category)
print(categories)
len(categories)

{'frog', 'airplane', 'can_opening', 'pouring_water', 'door_wood_knock', 'sea_waves', 'siren', 'laughing', 'glass_breaking', 'hen', 'breathing', 'hand_saw', 'clock_alarm', 'brushing_teeth', 'coughing', 'toilet_flush', 'crickets', 'sheep', 'clapping', 'washing_machine', 'door_wood_creaks', 'rooster', 'crow', 'train', 'church_bells', 'cow', 'keyboard_typing', 'engine', 'dog', 'thunderstorm', 'sneezing', 'drinking_sipping', 'chainsaw', 'vacuum_cleaner', 'clock_tick', 'chirping_birds', 'footsteps', 'water_drops', 'helicopter', 'mouse_click', 'fireworks', 'wind', 'crackling_fire', 'insects', 'snoring', 'car_horn', 'pig', 'rain', 'cat', 'crying_baby'}


50

In [9]:
dogs.values[3][0]

'1-30344-A-0.wav'

In [28]:
for i, dog in enumerate(dogs.values):
    dog_wav_path = "ESC-50-master/audio/" + dogs.values[i][0]
    print(dog_wav_path)
    ipd.display(ipd.Audio(url=dog_wav_path))

ESC-50-master/audio/1-100032-A-0.wav


ESC-50-master/audio/1-110389-A-0.wav


ESC-50-master/audio/1-30226-A-0.wav


ESC-50-master/audio/1-30344-A-0.wav


ESC-50-master/audio/1-32318-A-0.wav


...

ESC-50-master/audio/5-212454-A-0.wav


ESC-50-master/audio/5-213855-A-0.wav


ESC-50-master/audio/5-217158-A-0.wav


ESC-50-master/audio/5-231762-A-0.wav


ESC-50-master/audio/5-9032-A-0.wav
